 # FactSales – Silver → Gold

**Objetivo:**  
Crear la tabla de hechos `FactSales` en la capa Gold.

---

**Proceso:**  
1. Leer los pedidos de `Orders` desde la capa Silver.  
2. Convertir los campos de texto de fecha (`OrderDate`, `DueDate`, `ShipDate`) a tipo `date` para permitir su vinculación con `DimFecha`.  
3. Sustituir las fechas por sus respectivas claves foráneas (`OrderDateKey`, `DueDateKey`, `ShipDateKey`) obtenidas desde `DimFecha`.  
4. Resolver las claves foráneas con las dimensiones:  
   - **Cliente:** `DimCustomer`  
   - **Empleado:** `DimEmployee`  
   - **Producto:** `DimProduct`  
   - **Proveedor:** `DimVendor`  
   - **Categoría / Subcategoría:** `DimProductCategory`, `DimProductSubCategory`  
   - **Tienda:** `DimStore` (nueva relación a partir de `StoreID`)  
5. Calcular las métricas de negocio (`SubTotal`, `TaxAmt`, `Freight`, `TotalDue`, `OrderQty`, `UnitPrice`, `LineTotal`, etc.).  
6. Cargar el resultado consolidado en `Tables/FactSales` dentro de la capa Gold en formato **Delta**.

---


In [12]:
from pyspark.sql.functions import col, to_date

# Cargar tabla Orders desde Silver
df_orders = spark.table("AdventureWorks_SilverLayer.AdventureWorks_Silver_Orders")

# Conversión segura de fechas (texto → date)
df_orders = df_orders \
    .withColumn("OrderDate", to_date(col("OrderDate"), "M/d/yyyy")) \
    .withColumn("DueDate", to_date(col("DueDate"), "M/d/yyyy")) \
    .withColumn("ShipDate", to_date(col("ShipDate"), "M/d/yyyy"))

# Registrar vista temporal para usarla en SQL
df_orders.createOrReplaceTempView("vw_Orders_Converted")


# Crear vista FactSales
spark.sql("""
CREATE OR REPLACE TEMP VIEW tmpFactSales AS
SELECT
    o.SalesOrderID AS SalesOrderKey,
    IFNULL(c.CustomerKey, -1) AS CustomerKey,
    IFNULL(e.EmployeeKey, -1) AS EmployeeKey,
    IFNULL(p.ProductKey, -1) AS ProductKey,
    IFNULL(vp.VendorID, -1) AS VendorKey,
    IFNULL(psc.CategoryKey, -1) AS CategoryKey,
    IFNULL(psc.SubCategoryKey, -1) AS SubCategoryKey,
    IFNULL(s.StoreKey, -1) AS StoreKey,  -- 🔹 Nueva relación con DimStore

    -- Claves de fechas
    dfOrder.DateKey AS OrderDateKey,
    dfDue.DateKey   AS DueDateKey,
    dfShip.DateKey  AS ShipDateKey,

    -- Métricas de negocio
    o.SubTotal,
    o.TaxAmt,
    o.Freight,
    o.TotalDue,
    o.OrderQty,
    o.UnitPrice,
    o.UnitPriceDiscount,
    o.LineTotal,
    o.ETL_DateInserted

FROM vw_Orders_Converted o

-- Dimensiones existentes
LEFT JOIN AdventureWorks_GoldLayer.DimCustomer c       ON c.CustomerKey = o.CustomerID
LEFT JOIN AdventureWorks_GoldLayer.DimEmployee e       ON e.EmployeeKey = o.EmployeeID
LEFT JOIN AdventureWorks_GoldLayer.DimProduct p        ON p.ProductKey  = o.ProductID
LEFT JOIN AdventureWorks_GoldLayer.DimProductSubCategory psc ON psc.SubCategoryKey = p.SubCategoryID

-- Vendor (muchos a muchos)
LEFT JOIN (
    SELECT ProductID, VendorID
    FROM AdventureWorks_SilverLayer.AdventureWorks_Silver_VendorProduct
    GROUP BY ProductID, VendorID
) vp ON vp.ProductID = p.ProductKey

-- Relación con DimStore
LEFT JOIN AdventureWorks_GoldLayer.DimStore s ON s.StoreKey = o.StoreID

-- DimFecha
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfOrder ON dfOrder.FullDate = o.OrderDate
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfDue   ON dfDue.FullDate   = o.DueDate
LEFT JOIN AdventureWorks_GoldLayer.DimFecha dfShip  ON dfShip.FullDate  = o.ShipDate
""")

# Guardar FactSales en Gold
df = spark.sql("SELECT * FROM tmpFactSales")

# Verificar primeras filas antes de guardar
df.show(10)  

# Guardar la tabla FactSales en Gold (Delta)
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/FactSales")

StatementMeta(, a9a21c0d-11ab-474e-8210-11f53e5718de, 14, Finished, Available, Finished)

+-------------+-----------+-----------+----------+---------+-----------+--------------+--------+------------+----------+-----------+---------+--------+--------+---------+--------+---------+-----------------+---------+--------------------+
|SalesOrderKey|CustomerKey|EmployeeKey|ProductKey|VendorKey|CategoryKey|SubCategoryKey|StoreKey|OrderDateKey|DueDateKey|ShipDateKey| SubTotal|  TaxAmt| Freight| TotalDue|OrderQty|UnitPrice|UnitPriceDiscount|LineTotal|    ETL_DateInserted|
+-------------+-----------+-----------+----------+---------+-----------+--------------+--------+------------+----------+-----------+---------+--------+--------+---------+--------+---------+-----------------+---------+--------------------+
|        53451|       1803|        284|       965|       -1|          1|             3|    1422|    20130731|  20130812|   20130807| 17417689| 1762962|  550926| 19731577|       1|  3340575|               15|283948875|2025-10-04 02:36:...|
|        53459|        293|        276|     